In [7]:
import glob, os, sys
from pprint import pprint
import pickle
data_dir = 'data_files/'
### Let's first get a list of files from the datafiles directory that I may (or may not?? ) want to parse
dataFiles = glob.glob(data_dir+'20160*')

In [2]:
print dataFiles[20]

data_files/20160402


In [3]:
#!pip install plotly
import plotly
from plotly.graph_objs import Scatter, Layout, Bar
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [6]:
executed_jobs = 0
LogOutput = []
for f in dataFiles:
    #print f
    LogOutput.append( analyze_data_file( f) )

In [8]:
#print len(allExecutedJobs)  ### Only 30,000 jobs apparently were executed.. I must have deleted a lot of them?
#pprint( allExecutedJobs[2]) ### Just print the second record
import datetime as dt


totalWallTime = dt.timedelta(hours=0, minutes=0)
totalCpuTime =  dt.timedelta(hours=0, minutes=0)

for idx,row in enumerate(allExecutedJobs):
    ruWt = row['resourcesUsedWallTime']  
    #print row

    ruWtDt  = dt.datetime.strptime(ruWt,"%H:%M:%S")
    curJobWallTime = dt.timedelta(hours=ruWtDt.hour, minutes=ruWtDt.minute, seconds=ruWtDt.second)
    totalWallTime += curJobWallTime
    
    ###Lets do the same thing for CPUTime
    ruCpu = row['cpuTotal']
    ruCpuDt  = dt.datetime.strptime(ruCpu,"%H:%M:%S")
    curJobCpuTime = dt.timedelta(hours=ruCpuDt.hour, minutes=ruCpuDt.minute, seconds=ruCpuDt.second)
    totalCpuTime += curJobCpuTime
    #'cpuTotal': '02:27:06'
    


print totalWallTime,totalCpuTime

1363 days, 6:54:31 1759 days, 16:28:03


In [ ]:

pprint(LogOutput)

In [11]:
import plotly.plotly as py
import plotly.graph_objs as go

xdata = [ x['LogDate'].replace('data_files',' ')  for x in LogOutput]
ydata = [y['executed_jobs'] for y in LogOutput]

data = [go.Bar( x=xdata, y=ydata)]
iplot(data, filename='basic-bar')

In [5]:
import re

mfrSubmit=  re.compile(' .*user=(?P<user>.*) group=(?P<group>.*) jobname=(?P<jobname>.*) queue=(?P<queue>.*) ctime=(?P<ctime>\d+) qtime=(?P<qtime>\d+) \
etime=(?P<etime>\d+) start=(?P<start>\d+) owner=(?P<owner>.*) exec_host=(?P<exec_host>.*)' )
##my first regex

mfrExec = re.compile('.*;(?P<jobEntryType>.);.*user=(?P<user>.*) group=(?P<group>.*) jobname=(?P<jobname>.*) queue=(?P<queue>.*) ctime=(?P<ctime>\d+) qtime=(?P<qtime>\d+) \
etime=(?P<etime>\d+) start=(?P<start>\d+) owner=(?P<owner>.*) exec_host=(?P<exec_host>.*)' )


mfrExec2 = re.compile('.*;(?P<jobEntryType>.);.*user=(?P<user>.*) group=(?P<group>.*) jobname=(?P<jobname>.*) queue=(?P<queue>.*) ctime=(?P<ctime>\d+) qtime=(?P<qtime>\d+) etime=(?P<etime>\d+) start=(?P<start>\d+) owner=(?P<owner>.*) exec_host=(?P<exec_host>.*) Resource_List.neednodes=(?P<needNodes>\d+):ppn=(?P<needppn>\d+):(?P<needResourceName>.*) Resource_List.walltime=(?P<ndwalltime>.*) session=(?P<session>\d+) total_execution_slots=(?P<totExecSlots>\d+) unique_node_count=(?P<uniqueNodeCount>\d+) end=(?P<endtime>\d+) Exit_status=(?P<exitStatus>.) resources_used.cput=(?P<cpuTotal>.*) resources_used.mem=(?P<resourcesUsedMem>\d+kb) resources_used.vmem=(?P<resourcesedUsedVMem>\d+kb) resources_used.walltime=(?P<resourcesUsedWallTime>.*)')


allExecutedJobs = []

def analyze_data_file( fileName):
    """This will scan a log file from PBS/Torque and extract meaningful information about how many jobs were
    run, how long they took, etc"""

    executed_jobs = 0
    wall_time = 0 ### This really should be parsed... and probably load the data into a database
    
    with open(fileName, 'rU') as infile:
        for line in infile:
            line = line.strip()
            m = mfrExec2.search(line)
            if m:
                if m.groupdict()['jobEntryType'] == "E":
                    curJobInfo = m.groupdict()
                    executed_jobs +=1
                    allExecutedJobs.append(curJobInfo)

                    
                    
    return (  {'LogDate': fileName, 'executed_jobs': executed_jobs})
            

In [5]:
### Let's do some VERY basic analysis...

In [11]:
with open('allExecJobs.pickle', 'w') as fp:
    pickle.dump(allExecutedJobs,fp)

In [ ]:
sample_file = dataFiles[3]
with  open(sample_file,'rU') as fp:
    sampleData = fp.readlines()

In [ ]:
from pprint import pprint


mfr=  re.compile(' .*user=(?P<user>.*) group=(?P<group>.*) jobname=(?P<jobname>.*) queue=(?P<queue>.*) ctime=(?P<ctime>\d+) qtime=(?P<qtime>\d+) \
etime=(?P<etime>\d+) start=(?P<start>\d+) owner=(?P<owner>.*) exec_host=(?P<exec_host>.*) Resource_List.(?P<rsrc>.*) Resource_List.(?P<rsrc2>.*) \
Resource_List.(?P<rsrc3>.*)' )


mfr=  re.compile(' .*user=(?P<user>.*) group=(?P<group>.*) jobname=(?P<jobname>.*) queue=(?P<queue>.*) ctime=(?P<ctime>\d+) qtime=(?P<qtime>\d+) \
etime=(?P<etime>\d+) start=(?P<start>\d+) owner=(?P<owner>.*) exec_host=(?P<exec_host>.*) Resource_List.(?P<rsrc>.*) Resource_List.(?P<rsrc2>.*) \
Resource_List.(?P<rsrc3>.*)' )


###session=1417 total_execution_slots=2 unique_node_count=1 end=1458100531 Exit_status=0 resources_used.cput=00:00:17 resources_used.mem=2703476kb resources_used.vmem=4724852kb resources_used.walltime=00:01:08
### I can parse the above separtely

rsrcUsedList = re.compile('session=(?P<sessionID>.*) total_execution_slots=(?P<tExecSlots>\d+) unique_node_count=(?P<uNodeCount>\d+) \
end=(?P<end>\d+) Exit_status=(?P<ExitStatus>.*) resources_used.cput=(?P<cput>.*) resources_used.mem=(?P<memUsed>\d+kb) \
resources_used.vmem=(?P<vmem>\d+kb) resources_used.walltime=(?P<walltime>.*)')

sid = 5

print sampleData[sid]
# m= mfr.search(sampleData[sid])
# if m:
#     pprint( m.groupdict())

m2 = rsrcUsedList.search(sampleData[sid])
if m2:
    pprint (m2.groupdict())
                          

In [ ]:
for r in sampleData:
    m2 = rsrcUsedList.search(r)
    if m2:
        pprint(m2.groupdict())